## Image Caption Generator

YAPILACAKLAR

- dataset çok büyük, cache te tutabiliriz ( torch.save("filename.pt") şeklindeydi sanırım, bakalım ona ) ya da şimdi development aşamasında sadece bi kısmını alabiliriz (dataset = load_dataset("coco", split='train[:%10]')) gibi bişiydi bizde şu an  'dataset = load_dataset("coco", split='train[:%10]')' şeklinde. ya da başka bilmediğim bi yol varsa o da olur.

- Google Colab'de ya da herhangi biyerde çalışabilir hâle getirmek lazım, şu an bazı şeyler sorunlu olabilir, eklenmesi gereken şeyler olabilir

- Her method ne yapıyor nasıl çalışıyor iyice öğrenmek ve bunu nasıl geliştirebiliriz neler kullanabiliriz ya da neler eksik vs.
Bazı kavramlar yeni olacak, zorlanabilirsiniz, chatgpt ye soruyom ben de bolca :) ,gruba sorabilirsiniz, hallolcakkkk :)

Yapcazzz bu işiii <3


![image.png](attachment:image.png)


# New Section

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.datasets import CocoDetection
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import os
from PIL import Image
import numpy as np
import requests
from io import BytesIO
from datasets import load_dataset
from tqdm import tqdm
import cv2
import pycocoevalcap
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
from collections import Counter
# for metrics, COCO API metrics is used
import json
from pycocoevalcap import bleu, meteor, rouge, spice
#from pycocoevalcap.evalcap import COCOEvalCap

# Load YOLOv5 for feature extraction
import torch.hub

In [3]:
# setting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load pre-trained YOLOv5 model from Torch Hub
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).eval().to(device)

Using cache found in /home/pokepe/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-15 Python-3.10.0 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
# Define transformations for image preprocessing
# !!!!! NOTE chatgpt says this is not needed in yolov5
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
# Vocabulary, will be used later.
vocabulary = []

In [7]:
# "annotations": [{"image_id": 179765,"id": 38,"caption": "A black Honda motorcycle parked in front of a garage."},...}
class CocoDataset(Dataset): # <start> cat sat on the mat <end> -> {0 32 24 34 3 3 1 }
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.tokenizer = get_tokenizer("basic_english")  # Tokenizer from torchtext
        self.annotations = self.coco_data['annotations']
        # Build vocabulary
        self.vocab = self.build_vocab()
        self.image_id_to_file = {img['id']: img['file_name'] for img in self.coco_data['images']}
        self.image_id_to_img = {}


    def build_vocab(self):
        counter = Counter()
        for annotation in tqdm(self.annotations):
            caption = annotation['caption']
            tokens = self.tokenizer(caption.lower())
            counter.update(tokens)
        vocab = build_vocab_from_iterator([counter], specials=["<pad>", "<unk>", "<bos>", "<eos>"])
        vocab.set_default_index(vocab["<unk>"])  # Out-of-vocabulary words will be mapped to <unk>
        global vocabulary
        vocabulary = vocab
        return vocab

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_id = self.annotations[idx]['image_id']

        file_name = self.image_id_to_file.get(image_id)
        img_path = os.path.join(self.root_dir, file_name)

        try:
          image = Image.open(img_path).convert('RGB')
          self.image_id_to_img[image_id] = image # NOTE
        except Exception as e:
          print(f"Error opening image: {file_name}")
          raise e

        caption = self.annotations[idx]['caption']
        tokens = self.tokenizer(caption.lower())

        # Convert caption tokens to indices
        caption_indices = [self.vocab['<bos>']] + [self.vocab[token] for token in tokens] + [self.vocab['<eos>']]

        if self.transform:
            image = self.transform(image)

        return image, {'image_id': image_id,'captions': torch.tensor(caption_indices, dtype=torch.long)}


In [61]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        self.cnn = models.resnet50(pretrained=True).to(device)
        self.backbone = nn.Sequential(*list(self.cnn.children())[:-1])  # Get everything except the last detection layer
        self.fc = nn.Linear(self.cnn.fc.in_features, embed_size)  # Linear layer to resize

    def forward(self, images):
        with torch.no_grad():  # Freeze CNN parameters
            features = self.backbone(images)
        features = features.view(features.size(0), -1)  # Flatten
        features = self.fc(features)
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        # Embedding layer to convert words to a vector representation
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # Bidirectional LSTM: hidden size is doubled because of bidirectional=True
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        # Linear layer: multiply hidden_size by 2 because of the bidirectional LSTM
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, features, captions):
        # Convert captions to embeddings
        embeddings = self.embedding(captions)
        # Concatenate features and embeddings along the sequence dimension
        inputs = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        # Pass through LSTM
        lstm_out, _ = self.lstm(inputs)
        # Pass through fully connected layer to generate vocabulary scores
        output = self.fc(lstm_out)
        return output

class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs

In [93]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        self.cnn = models.resnet50(pretrained=True).to(device)
        self.backbone = nn.Sequential(*list(self.cnn.children())[:-1])  # Get everything except the last detection layer
        self.fc = nn.Linear(self.cnn.fc.in_features, embed_size)  # Linear layer to resize
        self.dropout = nn.Dropout(0.3)  # Add dropout to the encoder

    def forward(self, images):
        with torch.no_grad():  # Freeze CNN parameters
            features = self.backbone(images)
        features = features.view(features.size(0), -1)  # Flatten
        features = self.fc(features)
        features = self.dropout(features)  # Apply dropout to the feature vector
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, dropout_prob=0.3):
        super(DecoderRNN, self).__init__()
        # Embedding layer to convert words to a vector representation
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # Bidirectional LSTM with dropout
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        # Linear layer: multiply hidden_size by 2 because of the bidirectional LSTM
        self.fc = nn.Linear(hidden_size * 2, vocab_size)
        # Add a dropout layer after the LSTM output before the final linear layer
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, features, captions):
        # Convert captions to embeddings
        embeddings = self.embedding(captions)
        # Concatenate features and embeddings along the sequence dimension
        inputs = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        # Pass through LSTM
        lstm_out, _ = self.lstm(inputs)
        # Apply dropout after LSTM output
        lstm_out = self.dropout(lstm_out)
        # Pass through fully connected layer to generate vocabulary scores
        output = self.fc(lstm_out)
        return output

class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs


In [94]:
from torch.utils.data import Subset
import random 

train_image_number = 20000
val_image_number = 4000

train_images_path = './coco/train2017'
train_annotations_path = './coco/annotations/captions_train2017.json'
val_images_path = './coco/val2017'
val_annotations_path = './coco/annotations/captions_val2017.json'

# Load COCO Dataset (captions and images) using
train_dataset = CocoDataset(train_images_path, train_annotations_path, transform=transform)

# Example: Fetch an image and its caption
image, caption = train_dataset[0]
print(image)  # Prints the caption token indices

val_dataset = CocoDataset(val_images_path, val_annotations_path, transform=transform)

subset_indices = [i for i in range(4000)]

train_subset = Subset(train_dataset, subset_indices)
val_subset = Subset(val_dataset, subset_indices)


100%|██████████| 591753/591753 [00:03<00:00, 178491.92it/s]


tensor([[[2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         ...,
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891]],

        [[2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         ...,
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857]],

        [[2.64000, 2.64000, 2.64000,  ..., 2.64000, 2.64000, 2.64000],
         [2.64000, 2.64000, 2.64000,  ..., 2.

100%|██████████| 25014/25014 [00:00<00:00, 174450.28it/s]


In [95]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    """Collates a batch of (image, caption) pairs and pads captions."""
    images, targets = zip(*batch)
    image_ids = [target["image_id"] for target in targets]
    captions = [target["captions"] for target in targets]
    # Pad captions to have the same length in the batch
    padded_captions = pad_sequence(captions, batch_first=True, padding_value=0)  # <pad> token is 0

    # Stack images into a batch
    images = torch.stack(images, 0) # dim: The dimension along which to stack. If dim=0, it adds a new first dimension to the tensors.
   # NO NEED FOR padded_captions = torch.stack(padded_captions, 0) , padded_captions are already stacked

    return images, image_ids, padded_captions

In [96]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    for images, image_ids, captions in dataloader:
        #captions = targets['captions']
        images = images.to(device)
        captions = captions.to(device)
        #print(len(captions))
        #print(len(images))
        optimizer.zero_grad()
        #print("images.shape",images.shape)
        #print("captions.shape",captions.shape)
        outputs = model(images, captions[:, :-1])  # Exclude last token in captions

        batch_size, seq_len, vocab_size = outputs.size()
        #print("batch_size",batch_size)
        #print("seq_len",seq_len)
        #print("vocab_size",vocab_size)
        #print("outputs.shape before flattening",outputs.shape)
        outputs = outputs.view(-1, vocab_size)  # Flattens outputs
        captions = captions.view(-1)
        #captions = captions[:, 1:].contiguous().view(-1)# Flatten to (batch_size * seq_len)
        #print("outputs.shape after flattening",outputs.shape)
        #print("captions.shape",captions.shape)
        loss = criterion(outputs, captions)
        #print(f"Loss: {loss.item()}")
        # loss = criterion(outputs.view(-1, outputs.size(2)), # flattens  outputs
        #                 captions[:, 1:].contiguous().view(-1)
        #                 )  # Teacher forcing
        loss.backward()
        optimizer.step()
        # Print the current loss for monitoring the training progress
        print(f"Loss: {loss.item()}")

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, image_ids, captions in dataloader:
            #captions = targets['captions']
            images = images.to(device)
            captions = captions.to(device)

            #TODO DIMENSIONALITY PROBLEM
            outputs = model(images, captions[:, :-1])  # Exclude last token in captions
            batch_size, seq_len, vocab_size = outputs.size()
            outputs = outputs.view(-1, vocab_size)  # Flattens outputs
            captions = captions.view(-1)
            loss = criterion(outputs, captions)  # Teacher forcing
            total_loss += loss.item()

    return total_loss / len(dataloader)


In [97]:
# create a directory for checkpoints -> this can be used for storing state of the model in the midst of the training.
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok= True)

def save_checkpoint(model, optimizer, epoch, loss, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")


In [98]:
# Hyperparameters
embed_size = 256
hidden_size = 512
num_epochs = 5
learning_rate = 0.001
batch_size = 32
num_layers = 1

# Data Loaders
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Vocab
train_dataset.vocab = train_dataset.build_vocab()
# Instantiate Encoder, Decoder, and Model
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, hidden_size, len(train_dataset.vocab), num_layers=num_layers).to(device) # TODO
model = ImageCaptioningModel(encoder, decoder).to(device)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()



100%|██████████| 591753/591753 [00:03<00:00, 170929.47it/s]


In [99]:
# Training Loop
for epoch in tqdm(range(num_epochs)):
    train(model, train_loader, optimizer, criterion, device)

    print(f"Training for epoch {epoch+1}/{num_epochs} completed.")

    # Commented out - quicker
    #val_loss = evaluate(model, val_loader, criterion, device)
    #print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}")

# Save the trained model
torch.save(model.state_dict(), 'image_captioning_model.pth')

  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 10.35101318359375
Loss: 9.784751892089844
Loss: 9.178844451904297
Loss: 8.729475021362305
Loss: 6.728742599487305
Loss: 5.183103561401367
Loss: 5.143291473388672
Loss: 3.7683749198913574
Loss: 5.785114288330078
Loss: 5.335980415344238
Loss: 3.994569778442383
Loss: 1.942413330078125
Loss: 3.8383495807647705
Loss: 3.181079626083374
Loss: 3.6779022216796875
Loss: 2.467512369155884
Loss: 3.846390724182129
Loss: 2.906925678253174
Loss: 2.5546250343322754
Loss: 3.5961592197418213
Loss: 3.1338915824890137
Loss: 2.4959683418273926
Loss: 3.070277690887451
Loss: 3.02244234085083
Loss: 2.8820559978485107
Loss: 3.1101720333099365
Loss: 2.707646608352661
Loss: 3.0312631130218506
Loss: 3.2644946575164795
Loss: 3.4826037883758545
Loss: 3.0322232246398926
Loss: 2.1658718585968018
Loss: 2.851793050765991
Loss: 3.1049251556396484
Loss: 2.8047478199005127
Loss: 2.276203155517578
Loss: 2.872710943222046
Loss: 2.4055185317993164
Loss: 2.7039906978607178
Loss: 3.1587555408477783
Loss: 3.48046803474426

 20%|██        | 1/5 [01:07<04:30, 67.51s/it]

Loss: 1.5721864700317383
Training for epoch 1/5 completed.
Loss: 1.4563034772872925
Loss: 1.3461027145385742
Loss: 1.6205213069915771
Loss: 1.496802568435669
Loss: 1.5751144886016846
Loss: 1.2397340536117554
Loss: 1.475003957748413
Loss: 1.4807353019714355
Loss: 1.3961129188537598
Loss: 1.4355486631393433
Loss: 1.4427669048309326
Loss: 1.5866799354553223
Loss: 1.4424347877502441
Loss: 1.3035309314727783
Loss: 1.2858244180679321
Loss: 1.1093006134033203
Loss: 1.4218581914901733
Loss: 1.2699815034866333
Loss: 1.6323643922805786
Loss: 1.2858151197433472
Loss: 1.3039826154708862
Loss: 1.2799832820892334
Loss: 1.443668246269226
Loss: 1.015271782875061
Loss: 1.2399942874908447
Loss: 1.0244604349136353
Loss: 1.3766454458236694
Loss: 1.380370020866394
Loss: 1.2365856170654297
Loss: 1.5034898519515991
Loss: 1.3039827346801758
Loss: 1.2768632173538208
Loss: 1.0852046012878418
Loss: 1.4643020629882812
Loss: 1.0500121116638184
Loss: 1.4029009342193604
Loss: 1.3448104858398438
Loss: 1.4521673917770

 40%|████      | 2/5 [02:12<03:18, 66.15s/it]

...

Loss: 0.42448335886001587
Training for epoch 3/5 completed.
Loss: 0.379003643989563
Loss: 0.3121258616447449
Loss: 0.40862932801246643
Loss: 0.4157111942768097
Loss: 0.26162153482437134
Loss: 0.29034650325775146
Loss: 0.3005828261375427
Loss: 0.21421091258525848
Loss: 0.4536508023738861
Loss: 0.2727847099304199
Loss: 0.302873432636261
Loss: 0.2544087767601013
Loss: 0.259854793548584
Loss: 0.3809535503387451
Loss: 0.21821992099285126
Loss: 0.34212976694107056
Loss: 0.44980689883232117
Loss: 0.29365116357803345
Loss: 0.3211265206336975
Loss: 0.3005695044994354
Loss: 0.41192013025283813
Loss: 0.31960776448249817
Loss: 0.3030775487422943
Loss: 0.2758401036262512
Loss: 0.25725528597831726
Loss: 0.30044040083885193
Loss: 0.36730700731277466
Loss: 0.31681084632873535
Loss: 0.22530122101306915
Loss: 0.3238416910171509
Loss: 0.3822339177131653
Loss: 0.2555922269821167
Loss: 0.3223743438720703
Loss: 0.35252565145492554
Loss: 0.2631235718727112
Loss: 0.3467898666858673
Loss: 0.2559923827648163
Lo

 80%|████████  | 4/5 [04:20<01:04, 64.45s/it]

Loss: 0.18351337313652039
Training for epoch 4/5 completed.
Loss: 0.1381349414587021
Loss: 0.13530279695987701
Loss: 0.10464068502187729
Loss: 0.11917698383331299
Loss: 0.1322881430387497
Loss: 0.09613919258117676
Loss: 0.13934239745140076
Loss: 0.06408267468214035
Loss: 0.1169775202870369
Loss: 0.07461756467819214
Loss: 0.12378624826669693
Loss: 0.12864723801612854
Loss: 0.09331797808408737
Loss: 0.08902929723262787
Loss: 0.13570448756217957
Loss: 0.12833160161972046
Loss: 0.09594336897134781
Loss: 0.10682720690965652
Loss: 0.08842155337333679
Loss: 0.09808490425348282
Loss: 0.11636833101511002
Loss: 0.13443325459957123
Loss: 0.1158982589840889
Loss: 0.1194649413228035
Loss: 0.12093605846166611
Loss: 0.09782528877258301
Loss: 0.11449678987264633
Loss: 0.136025071144104
Loss: 0.11077828705310822
Loss: 0.1303311139345169
Loss: 0.0995551347732544
Loss: 0.11035672575235367
Loss: 0.17065542936325073
Loss: 0.1356392353773117
Loss: 0.12095650285482407
Loss: 0.09689071029424667
Loss: 0.115411

100%|██████████| 5/5 [05:25<00:00, 65.19s/it]

Loss: 0.11968249082565308
Training for epoch 5/5 completed.


In [100]:
def generate_caption(image, model, vocab, device):

    model.eval()
    print(image.shape)
    image = image.unsqueeze(0).to(device) # unsqueezing for encapsulating it inside a batch
    print(image.shape)
    features = model.encoder(image)

    # Generate caption
    caption = ['<bos>']
    for _ in range(50):  # Maximum caption length
        input_caption = torch.tensor([vocab[token] for token in caption]).unsqueeze(0).to(device)
        outputs = model.decoder(features, input_caption)
        _, predicted = outputs.max(2)  # outputs.shape = (batch,seq_len,vocab_size), vocab_size needed because we need to calculate each word's probabilty and pick the best of them
        predicted_word = vocab.get_itos()[predicted[0, -1].item()] # get the first batch (only one), last predicted word in lstm(next word) convert it to string instead of an id
        caption.append(predicted_word)
        if predicted_word == '<eos>':
            break
    print(caption)
    return ' '.join(caption[1:-1])  # Removing <bos> and <eos>

# Load trained model and generate a caption
model.load_state_dict(torch.load('image_captioning_model.pth'))
#image = Image.open("path_to_image.jpg")  # Replace with your image path
#caption = generate_caption(image, model, train_dataset.vocab, device)
#print(f"Generated Caption: {caption}")


<All keys matched successfully>

In [101]:
def generate_captions_for_coco(val_loader,model,vocab, max_length= 50, num_samples=5):
    all_captions = {}
    model.eval()
    with torch.no_grad():
        for i, (images, image_ids,captions) in enumerate(val_loader):
            #image_ids = targets['image_id']
            images = images.to(device)
            for image_id, image in zip(image_ids,images):
              generated_caption = generate_caption(image, model, vocab, device)
              print(f"type(generated_caption) = {type(generated_caption)} , {type(image_id)}")
              all_captions[image_id] = generated_caption
              print(f"Generated Caption for Image {image_id}: {generated_caption}")
            if i >= num_samples:
                break
    print(all_captions)
    return all_captions



In [76]:

def evaluate_captioning_model(generated_captions, coco_annotation_file=val_annotations_path,coco_image_dir=val_images_path):
    """
    Evaluate the image captioning model using COCO evaluation metrics: BLEU, METEOR, ROUGE, CIDEr.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.

    Returns:
        dict: Dictionary containing BLEU, METEOR, ROUGE, CIDEr scores.
    """
    # Load the COCO dataset annotations (reference captions)
    coco = COCO(coco_annotation_file)

    # Create a dictionary for the generated captions (image_id -> caption)
    coco_results = [{'image_id': image_id, 'caption': caption} for image_id, caption in generated_captions.items()]


    # Save the generated captions in a temporary file
    with open('generated_captions.json', 'w') as f:
        json.dump(coco_results, f)

    # Load the results into COCO's evaluation API
    coco_results = coco.loadRes('generated_captions.json')
    print(coco_results)
    print("Generated Captions Image IDs:", generated_captions.keys())
    print("Ground Truth Image IDs:", coco.getImgIds())


    # since we filtered the images to contain only first 1000 images, lets filter the metric
    all_image_ids = coco.getImgIds()
    filtered_image_ids = [image_id for image_id in generated_captions.keys()]

    # we need to revise the filtered version of the actual annotations
    gts = {}
    for image_id in filtered_image_ids:
        caption_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(caption_ids)
        gts[image_id] = [annotation['caption'] for annotation in annotations]


    # Set up the evaluation
    print("gts.keys(): ",gts.keys())
    print("coco_results.keys(): ", generated_captions.keys())
    #assert(gts.keys() == coco_results.keys())

    coco_eval = COCOEvalCap(coco, coco_results)
    coco_eval.params['image_id'] = filtered_image_ids
    coco_eval.evaluate()

    # Extract and return the metrics (BLEU, METEOR, ROUGE, CIDEr)
    metrics = coco_eval.eval
    return metrics

generated_captions = generate_captions_for_coco(val_loader,model,vocabulary)
# Evaluate the generated captions
metrics = evaluate_captioning_model(generated_captions)


print("Evaluation Metrics:", metrics)

torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'mixture', 'giraffes', 'lounging', 'lounging', 'cutting', 'bread', 'bread', 'jelly', 'jelly', 'together', 'together', 'together', 'them', 'them', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: mixture giraffes lounging lounging cutting bread bread jelly jelly together together together them them
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'mixture', 'giraffes', 'lounging', 'lounging', 'cutting', 'bread', 'bread', 'jelly', 'jelly', 'together', 'together', 'together', 'them', 'them', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: mixture giraffes lounging lounging cutting bread bread jelly jelly together together together them them
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'mixture', 'giraffes', 'lounging', 'lounging', 'cutting', 'bread', 'bread', 'jelly', 'jelly', 'together

PTBTokenizer tokenized 2696 tokens at 40784.09 tokens per second.
PTBTokenizer tokenized 674 tokens at 12938.90 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 630, 'reflen': 547, 'guess': [630, 585, 540, 495], 'correct': [3, 0, 0, 0]}
ratio: 1.1517367458845489
Bleu_1: 0.005
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.007
computing Rouge score...
ROUGE_L: 0.005
computing CIDEr score...
CIDEr: 0.001
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.9

SPICE evaluation took: 8.316 s
SPICE: 0.000
Evaluation Metrics: {'Bleu_1': 0.004761904761897205, 'Bleu_2': 9.02219936599791e-11, 'Bleu_3': 2.4702658418915266e-13, 'Bleu_4': 1.3210111098846717e-14, 'METEOR': 0.006533338122938359, 'ROUGE_L': 0.004979034901080625, 'CIDEr': 0.0013499031851597159, 'SPICE': 0.0}


In [102]:
import matplotlib.pyplot as plt
from PIL import Image
from pycocotools.coco import COCO

def evaluate_captioning_model(generated_captions, coco_annotation_file=val_annotations_path, coco_image_dir=val_images_path):
    """
    Evaluate the image captioning model using COCO evaluation metrics: BLEU, METEOR, ROUGE, CIDEr.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.

    Returns:
        dict: Dictionary containing BLEU, METEOR, ROUGE, CIDEr scores.
    """
    # Load the COCO dataset annotations (reference captions)
    coco = COCO(coco_annotation_file)

    # Create a dictionary for the generated captions (image_id -> caption)
    coco_results = [{'image_id': image_id, 'caption': caption} for image_id, caption in generated_captions.items()]

    # Save the generated captions in a temporary file
    with open('generated_captions.json', 'w') as f:
        json.dump(coco_results, f)

    # Load the results into COCO's evaluation API
    coco_results = coco.loadRes('generated_captions.json')
    print(coco_results)
    print("Generated Captions Image IDs:", generated_captions.keys())
    print("Ground Truth Image IDs:", coco.getImgIds())

    # Filter the image IDs for evaluation
    filtered_image_ids = [image_id for image_id in generated_captions.keys()]

    # Get the reference annotations for evaluation
    gts = {}
    for image_id in filtered_image_ids:
        caption_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(caption_ids)
        gts[image_id] = [annotation['caption'] for annotation in annotations]

    # Set up the evaluation
    print("gts.keys(): ", gts.keys())
    print("coco_results.keys(): ", generated_captions.keys())

    coco_eval = COCOEvalCap(coco, coco_results)
    coco_eval.params['image_id'] = filtered_image_ids
    coco_eval.evaluate()

    # Extract and return the metrics (BLEU, METEOR, ROUGE, CIDEr)
    metrics = coco_eval.eval
    return metrics

def show_images_with_captions(generated_captions, coco_annotation_file, coco_image_dir):
    """
    Display images with their generated captions.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.
        coco_image_dir (str): Path to the directory containing COCO images.
    """
    # Load the COCO dataset annotations
    coco = COCO(coco_annotation_file)

    for image_id, caption in generated_captions.items():
        # Get image information from COCO
        try:
            image_info = coco.loadImgs(image_id)[0]
            image_path = f"{coco_image_dir}/{image_info['file_name']}"
            print(f"Image path: {image_path}")  # Debug: Print the image path
        except KeyError:
            print(f"Image ID {image_id} not found in the dataset.")
            continue

        # Load and display the image using PIL and matplotlib
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image file not found at {image_path}")
            continue

        # Print the caption
        print(f"Generated Caption: {caption}")

        # Display the image
        plt.figure(figsize=(8, 8))
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Generated Caption: {caption}", fontsize=14, wrap=True)
        plt.show()

# Generate captions for COCO validation images
generated_captions = generate_captions_for_coco(val_loader, model, vocabulary)

# Evaluate the generated captions
metrics = evaluate_captioning_model(generated_captions)

# Print evaluation metrics
print("Evaluation Metrics:", metrics)

# Display images and their captions
show_images_with_captions(generated_captions, val_annotations_path, val_images_path)


torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'installing', 'equipment', 'equipment', 'amongst', 'amongst', 'ruins', 'ruins', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: installing equipment equipment amongst amongst ruins ruins <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 

PTBTokenizer tokenized 2696 tokens at 50511.38 tokens per second.
PTBTokenizer tokenized 2249 tokens at 45727.87 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 2205, 'reflen': 577, 'guess': [2205, 2160, 2115, 2070], 'correct': [0, 0, 0, 0]}
ratio: 3.8214904679309853
Bleu_1: 0.000
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.000
computing Rouge score...
ROUGE_L: 0.000
computing CIDEr score...
CIDEr: 0.000
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [1.0

SPICE evaluation took: 20.67 s
SPICE: 0.000
Evaluation Metrics: {'Bleu_1': 4.535147392288193e-19, 'Bleu_2': 4.582144993566359e-19, 'Bleu_3': 4.630299619453005e-19, 'Bleu_4': 4.679659287209122e-19, 'METEOR': 0.0, 'ROUGE_L': 0.0, 'CIDEr': 0.0, 'SPICE': 0.0}
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Image path: ./coco/val2017/000000179765.jpg
Generated Caption: installing equipment equipment amongst amongst ruins ruins <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Image path: ./coco/val2017/000000190236.jpg
Generated Caption: installing equipment equipment amongst amongst ruins ruins <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa